# Imports

In [5]:
%pdb off
%load_ext autoreload
%autoreload 2
import sys
import importlib
from pathlib import Path

import numpy as np

# required to enable non-blocking interaction:
# %gui qt
# !env QT_API="pyqt5"
%gui qt5
# %gui qt6
# from PyQt5.Qt import QApplication
# # start qt event loop
# _instance = QApplication.instance()
# if not _instance:
#     _instance = QApplication([])
# app = _instance

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer

from enum import Enum

class SessionBatchProgress(Enum):
    """Indicates the progress state for a given session in a batch processing queue """
    NOT_STARTED = "NOT_STARTED"
    RUNNING = "RUNNING"
    COMPLETED = "COMPLETED"
    ABORTED = "ABORTED"

session_batch_status = {}
enable_saving_to_disk = False

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership # needed for batch_programmatic_figures
from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs, find_local_session_paths, batch_load_session

def build_eloy_computation_configs(sess, **kwargs):
    """ OPTIONALLY can be overriden by implementors to provide specific filter functions """
    # (4.0, 4.0)cm bins, (6.0, 6.0)cm gaussian smoothing
    # peak frate > 2Hz 
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(6.0, 6.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(2.5, 2.5), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(0.2, 0.2), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(0.2, 0.2), frate_thresh=0.2, time_bin_size=0.025, computation_epochs = None), spike_analysis=None)]


def batch_programmatic_figures(curr_active_pipeline):
    """ programmatically generates and saves the batch figures 2022-12-07 

    """
    ## 🗨️🟢 2022-10-26 - Jonathan Firing Rate Analyses
    # Perform missing global computations                                                                                  #
    curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_jonathan_replay_firing_rate_analyses', '_perform_short_long_pf_overlap_analyses'], fail_on_exception=True, debug_print=True)

    ## Get global 'jonathan_firing_rate_analysis' results:
    curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
    neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']

    # ==================================================================================================================== #
    # Batch Output of Figures                                                                                              #
    # ==================================================================================================================== #
    ## 🗨️🟢 2022-11-05 - Pho-Jonathan Batch Outputs of Firing Rate Figures
    # %matplotlib qt
    short_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY]
    short_only_aclus = short_only_df.index.values.tolist()
    long_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY]
    long_only_aclus = long_only_df.index.values.tolist()
    shared_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED]
    shared_aclus = shared_df.index.values.tolist()
    print(f'shared_aclus: {shared_aclus}')
    print(f'long_only_aclus: {long_only_aclus}')
    print(f'short_only_aclus: {short_only_aclus}')

    active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'


    # ==================================================================================================================== #
    # Output Figures to File                                                                                               #
    # ==================================================================================================================== #
    ## PDF Output
    # %matplotlib qtagg
    import matplotlib
    # configure backend here
    # matplotlib.use('Qt5Agg')
    # backend_qt5agg
    matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

    n_max_page_rows = 10
    _batch_plot_kwargs_list = _build_batch_plot_kwargs(long_only_aclus, short_only_aclus, shared_aclus, active_identifying_session_ctx, n_max_page_rows=n_max_page_rows)
    active_out_figures_list = _perform_batch_plot(curr_active_pipeline, _batch_plot_kwargs_list, figures_parent_out_path=None, write_pdf=False, write_png=True, progress_print=True, debug_print=False)

    return active_identifying_session_ctx, active_out_figures_list


global_data_root_parent_path = Path(r'W:\Data') # Windows Apogee
# global_data_root_parent_path = Path(r'/media/MAX/Data') # Diba Lab Workstation Linux
# global_data_root_parent_path = Path(r'/Volumes/MoverNew/data') # rMBP
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Pipeline

In [6]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

# %%viztracer
active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

## Animal `gor01`:
local_session_parent_path = local_session_root_parent_path.joinpath('gor01', 'one')
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

# local_session_parent_path = local_session_root_parent_path.joinpath('gor01', 'two')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])

# ### Animal `vvp01`:
# local_session_parent_path = local_session_root_parent_path.joinpath('vvp01', 'one')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_path = local_session_root_parent_path.joinpath('vvp01', 'two')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_paths_list = [local_session_parent_path.joinpath(a_name).resolve() for a_name in local_session_names_list]

## Initialize `session_batch_status` with the NOT_STARTED status if it doesn't already have a different status
for curr_session_basedir in local_session_paths_list:
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status is None:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED # set to not started if not present
session_batch_status

local_session_names_list: ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']


{WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_3-23-37'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-13_14-42-6'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>}

In [7]:
## Run batch queue
for curr_session_basedir, curr_session_name in zip(local_session_paths_list, local_session_names_list):
    print(f'curr_session_basedir: {str(curr_session_basedir)}')
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status.name != SessionBatchProgress.COMPLETED.name:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED
        try:
            session_batch_status[curr_session_basedir] = SessionBatchProgress.RUNNING
            curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, curr_session_basedir, force_reload=False)
            active_identifying_session_ctx, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)
            session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED
            print(f'completed processing for {curr_session_basedir}: {active_identifying_session_ctx}')
        except Exception as e:
            print(f'ERROR processing {curr_session_basedir} with error {e}')
            session_batch_status[curr_session_basedir] = SessionBatchProgress.ABORTED
            # raise e
    else:
        print(f'\t already completed')

print(f'session_batch_status: {session_batch_status}')
print('!!! done running batch !!!')



curr_session_basedir: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53
finalized_loaded_sess_pickle_path: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\loadedSessPickle.pkl
Loading loaded session pickle file results to W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
global_named_timerange: <NamedTimerange: {'name': 'maze', 'start_end_times': [22.26, 1932.4200048116618]};>, first_included_epoch_name: maze1, last_included_epoch_name: maze2
active_session_filter_configurations: {'maze1': <function DataSessionFormatBaseRegisteredClass.build_default_filter_functions.<locals>.<dictcomp>.<lambda> at 0x000001EF73D21DC0>, 'maze2': <function DataSessionFormatBaseRegisteredClass.build_default_filter_functions.<locals>.<dictcomp>.<lambda> at 0x000001EF73D21E50>, 'maze': <function DataSessionFormatBaseRegisteredClass.build_g

c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-12-07\kdiba_2006-6-07_11-26-53_batch_plot_test_long_only_[17, 18, 21, 24, 61, 62].png
include_whitelist: ['maze1_PYR', 'maze2_PYR', 'maze_PYR', 'maze1', 'maze2', 'maze']
long_epoch_name: maze1_PYR, short_epoch_name: maze2_PYR, global_epoch_name: maze


c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-12-07\kdiba_2006-6-07_11-26-53_batch_plot_test_shared_1of6_[2, 3, 4, 5, 6, 7, 8, 9, 10, 11].png
include_whitelist: ['maze1_PYR', 'maze2_PYR', 'maze_PYR', 'maze1', 'maze2', 'maze']
long_epoch_name: maze1_PYR, short_epoch_name: maze2_PYR, global_epoch_name: maze


...

	 saved C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-12-07\kdiba_2006-6-13_14-42-6_batch_plot_test_shared_4of6_[34, 35, 36, 38, 39, 40, 41, 42, 43, 44].png
include_whitelist: ['maze1_PYR', 'maze2_PYR', 'maze_PYR', 'maze1', 'maze2', 'maze']
long_epoch_name: maze1_PYR, short_epoch_name: maze2_PYR, global_epoch_name: maze


c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-12-07\kdiba_2006-6-13_14-42-6_batch_plot_test_shared_5of6_[45, 47, 48, 49, 50, 51, 52, 53, 54, 55].png
include_whitelist: ['maze1_PYR', 'maze2_PYR', 'maze_PYR', 'maze1', 'maze2', 'maze']
long_epoch_name: maze1_PYR, short_epoch_name: maze2_PYR, global_epoch_name: maze


c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-12-07\kdiba_2006-6-13_14-42-6_batch_plot_test_shared_6of6_[56, 57, 58, 59, 60].png
completed processing for W:\Data\KDIBA\gor01\one\2006-6-13_14-42-6: kdiba_2006-6-13_14-42-6
session_batch_status: {WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53'): <SessionBatchProgress.COMPLETED: 'COMPLETED'>, WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15'): <SessionBatchProgress.COMPLETED: 'COMPLETED'>, WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43'): <SessionBatchProgress.COMPLETED: 'COMPLETED'>, WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_3-23-37'): <SessionBatchProgress.ABORTED: 'ABORTED'>, WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31'): <SessionBatchProgress.COMPLETED: 'COMPLETED'>, WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-13_14-42-6'): <SessionBatchProgress.COMPLETED: 'COMPLETED'>}
!!! done running batch !!!


# 🟢 Single basedir (non-batch) testing:

In [ ]:
basedir = local_session_paths_list[4] # NOT 3
print(f'basedir: {str(basedir)}')

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir)


## 🗨️ 2022-10-26 - Jonathan Firing Rate Analyses

In [ ]:
# ==================================================================================================================== #
# Perform missing global computations                                                                                  #
# ==================================================================================================================== #
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_jonathan_replay_firing_rate_analyses', '_perform_short_long_pf_overlap_analyses'], fail_on_exception=True, debug_print=True)

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
short_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY]
short_only_aclus = short_only_df.index.values.tolist()
long_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY]
long_only_aclus = long_only_df.index.values.tolist()
shared_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED]
shared_aclus = shared_df.index.values.tolist()
print(f'shared_aclus: {shared_aclus}')
print(f'long_only_aclus: {long_only_aclus}')
print(f'short_only_aclus: {short_only_aclus}')

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

## 🗨️ 2022-11-05 - Pho-Jonathan Batch Outputs of Firing Rate Figures
====================================================================================================================
Batch Output of Figures                                                                                             
====================================================================================================================

In [ ]:
# %matplotlib qt

# ==================================================================================================================== #
# Output Figures to File                                                                                               #
# ==================================================================================================================== #
## PDF Output
# %matplotlib qtagg
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

n_max_page_rows = 10
_batch_plot_kwargs_list = _build_batch_plot_kwargs(long_only_aclus, short_only_aclus, shared_aclus, active_identifying_session_ctx, n_max_page_rows=n_max_page_rows)
active_out_figures_list = _perform_batch_plot(curr_active_pipeline, _batch_plot_kwargs_list, figures_parent_out_path=None, write_pdf=False, write_png=True, progress_print=True, debug_print=False)

## Individual Plotting Tests:

### Common Config

In [ ]:
# display matplotlib figures in a qt backed window. Works in Jupyter notebooks
from neuropy.utils.matplotlib_helpers import enumTuningMap2DPlotVariables # for getting the variant name from the dict

from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D, capture_new_figures_decorator
fig_man = PhoActiveFigureManager2D(name=f'fig_man') # Initialize a new figure manager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper


## PDF Output, NOTE this is single plot stuff: uses active_config_name
from matplotlib.backends import backend_pdf
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

# # ## OPTIONS/SETTINGS:
# # enable_saving_to_disk = True
# # # enable_saving_to_disk = False
# # # open_folders_in_system_teafile_manager = True
# # open_folders_in_system_file_manager = False

# ## Old Individual Plotting Function Configs, still used as of 2022-08-10:
# # figure_format_config = dict(subplots=(20, 8), max_screen_figure_size=(2256, 2048)) # bad way
# # figure_format_config = dict(subplots=(None, 8), max_screen_figure_size=(None, 1868)) # good way
# figure_format_config = dict(subplots=(None, 8), max_screen_figure_size=(None, None)) # good way
# # figure_format_config = dict(subplots=(20, 8), max_screen_figure_size=(None, 1868))

# # figure_format_config = dict(subplots=(6, 6), max_screen_figure_size=(2256, 2048)) # bad way
# # RowColTuple(subplots[0], subplots[1])

# ## Add explicit column/row widths to fix window sizing issue:
# # figure_format_config = (dict(fig_column_width=1.0, fig_row_height=1.0) | figure_format_config)
# figure_format_config = (dict(fig_column_width=2.0, fig_row_height=2.0) | figure_format_config)
# # figure_format_config = (dict(fig_column_width=5.0, fig_row_height=5.0) | figure_format_config)

# ## Add additional default options:
# figure_format_config = (dict(enable_spike_overlay=False, debug_print=True, enable_saving_to_disk=enable_saving_to_disk) | figure_format_config)
# figure_format_config

# NOTE: if you want the outputs to be rendered to the same PDF, you must wrap all the plotting in the backend_pdf context:
def wrap_display_function_to_save_pdf(curr_display_function_name='_display_normal', filter_name='maze'):
    """ implicitly captures: active_identifying_filtered_session_ctx, _get_curr_figure_format_config 
    
    Usage:
        wrap_display_function_to_save_pdf(curr_display_function_name='_display_normal', filter_name=active_config_name)
    """
    built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name) # but this is specific to a given display function, right?
    active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name=curr_display_function_name)
    figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
    figure_format_config['enable_saving_to_disk'] = False # don't use the in-built figure export/saving to disk functionality as we want to wrap the output figure with the Pdf saving, not write to a .png
    with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
        # set_rasterized,  Axes.set_rasterization_zorder
        # fig0, figList1 = curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk)
        ## TODO: main display function call done here:
        out_figures_list = curr_active_pipeline.display(curr_display_function_name, filter_name, **figure_format_config)
        if not isinstance(out_figures_list, (list, tuple)):
            # not a list, just a scalar object
            plots = [out_figures_list] # make a single-element list
        else:
            # it is a list
            if len(out_figures_list) == 2:
                fig0, figList1 = out_figures_list # unpack
                plots = [fig0, *figList1]
            else:
                # otherwise just try and set the plots to the list
                plots = out_figures_list
                
        for i, a_fig in enumerate(plots):
            pdf.savefig(a_fig, transparent=True)
            pdf.attach_note(f'Page {i + 1}') 

            

### Single (Session, Filter) Context Plotting:

#### Utility:

In [ ]:
# Reload display functions:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.registered_display_function_names # ['_display_1d_placefield_validations', '_display_2d_placefield_result_plot_ratemaps_2D', '_display_2d_placefield_result_plot_raw', '_display_normal', '_display_placemaps_pyqtplot_2D', '_display_decoder_result', '_display_plot_most_likely_position_comparisons', '_display_two_step_decoder_prediction_error_2D', '_display_two_step_decoder_prediction_error_animated_2D', '_display_spike_rasters_pyqtplot_2D', '_display_spike_rasters_pyqtplot_3D', '_display_spike_rasters_pyqtplot_3D_with_2D_controls', '_display_spike_rasters_vedo_3D', '_display_spike_rasters_vedo_3D_with_2D_controls', '_display_spike_rasters_window', '_display_speed_vs_PFoverlapDensity_plots', '_display_3d_image_plotter', '_display_3d_interactive_custom_data_explorer', '_display_3d_interactive_spike_and_behavior_browser', '_display_3d_interactive_tuning_curves_plotter']
print(curr_active_pipeline.registered_display_function_names)

In [ ]:
## Test getting figure save paths:
_test_fig_path = curr_active_config.plotting_config.get_figure_save_path('test')
print(f'_test_fig_path: {_test_fig_path}\n\t exists? {_test_fig_path.exists()}')

In [ ]:
%matplotlib --list 
# Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']

In [ ]:
%matplotlib qt
## NOTE THAT ONCE THIS IS SET TO qt, it cannot be undone!

In [ ]:
fig_man.close_all()

In [ ]:
extant_figs_dict = fig_man.figures_dict
extant_fig_nums = fig_man.figure_nums
fig_num_figure_still_exists = [plt.fignum_exists(a_fig_num) for a_fig_num in extant_fig_nums]
fig_num_figure_still_exists #     # Figure is still opened or is closed
# if plt.fignum_exists(<figure number>):
#     # Figure is still opened
# else:
#     # Figure is closed
extant_figs_dict

# mpl.rcParams['toolbar'] = 'None'

In [ ]:
fig_man.reshow_figure(extant_figs_dict[10])

#### 🐞 Planning Helper Dock

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockPlanningHelperWidget.DockPlanningHelperWidget import DockPlanningHelperWidget

In [ ]:
test_dock_planning_widget = DockPlanningHelperWidget(dock_title='Test Dock Widget 1', dock_id='Test Dock Widget 1')
win.add_display_dock(identifier=test_dock_planning_widget.identifier, widget=test_dock_planning_widget)

In [ ]:
# test_dock_planning_widget.parent()
test_dock_planning_widget.embedding_dock_item

In [ ]:
test_dock_planning_widget.embedding_dock_item.area # DockArea 
# test_dock_planning_widget.embedding_dock_item.orig_area # DockArea 

In [ ]:
test_dock_planning_widget.embedding_dock_item.setOrientation

In [ ]:
test_dock_planning_widget, test_dock_item = win.create_planning_helper_dock()
test_dock_planning_widget

In [ ]:
win.ui

In [ ]:
test_dock_planning_widget2, test_dock_item2 = win.create_planning_helper_dock()
test_dock_planning_widget2

In [ ]:
# curr_display_dock_planning_helper_widgets = win.displayDockArea.findChildren(DockPlanningHelperWidget, QtCore.QRegExp(".*"), QtCore.Qt.FindChildrenRecursively)
# curr_display_dock_planning_helper_widgets

curr_display_dock_planning_helper_widgets = win.displayDockArea.findChildren(DockPlanningHelperWidget, QtCore.QRegExp(".*"), QtCore.Qt.FindChildrenRecursively)
# curr_display_dock_planning_helper_widgets
# win.connection_man
for a_child_helper_widget in curr_display_dock_planning_helper_widgets:
    a_child_helper_widget.action_create_new_dock.connect(perform_create_new_relative_dock)

In [ ]:
win.dynamic_display_dict

In [ ]:
win.dynamic_display_dict['Test Dock Widget 1']



# "dock"
# "widget"

In [ ]:
from pyphoplacecellanalysis.External.pyqtgraph.dockarea.Dock import Dock

# curr_display_dock_items = win.displayDockArea.children()
# curr_display_dock_items = win.displayDockArea.findChildren(Dock)
curr_display_dock_items = win.displayDockArea.findChildren(Dock, QtCore.QRegExp(".*"), QtCore.Qt.FindChildrenRecursively)
curr_display_dock_items

In [ ]:
win.find_display_dock('Test Dock Widget 1')

In [ ]:
dock_item_titles = [a_dock_item.title() for a_dock_item in curr_display_dock_items]
dock_item_titles
# [a_dock_item for a_dock_item in curr_display_dock_items]

In [ ]:
first_dock_item = curr_display_dock_items[0]

In [ ]:
first_dock_item.title() # 
# first_dock_item.setTitle
first_dock_item.geometry() # PyQt5.QtCore.QRect(0, 792, 842, 786)

In [ ]:
first_dock_item.frameGeometry() # PyQt5.QtCore.QRect(0, 792, 842, 786)
first_dock_item.widgets # list, e.g. [<pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockPlanningHelperWidget.DockPlanningHelperWidget.DockPlanningHelperWidget at 0x1d6b2a1b820>]
first_dock_item.name()


### Systematic Display Function Testing

#### Matplotlib-based plots:

In [ ]:
import matplotlib
# matplotlib.use('AGG') # non-interactive backend
# %matplotlib -l

matplotlib.use('Qt5Agg') # non-interactive backend
## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

In [ ]:
curr_active_pipeline.filtered_session_names # ['maze', 'sprinkle']
active_config_name = 'maze'

In [ ]:
# active_display_to_pdf_fn = wrap_display_function_to_save_pdf
active_display_to_pdf_fn = programmatic_display_to_PDF

In [ ]:
%%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations', filter_name=active_config_name) # 🟢✅ Working

In [ ]:
# %%capture
# active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_raw', filter_name=curr_active_pipeline.filtered_contexts.maze) # 🟢✅ Working
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_raw') # 🟢✅ Working


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D') # , filter_name=active_config_name # 🟢 Mostly Working # works but now only displays a single filter (where before it displayed all of them and rendered them to individual pages of the PDF)
# BROKEN: TypeError: neuropy.utils.debug_helpers.safely_accepts_kwargs.<locals>._safe_kwargs_fn() got multiple values for keyword argument 'computation_config'



In [ ]:
%%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_normal', filter_name=active_config_name) # 🟢✅ Working

### 🐞👁️‍🗨️🔜 TODO: FINISH THIS UP AND FIGURE OUT WHATEVER THE HELL I'M DOING HERE

In [ ]:
curr_display_function_name = '_display_2d_placefield_result_plot_ratemaps_2D'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
out_fig_list = []
active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name=curr_display_function_name)
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
figure_format_config['enable_saving_to_disk'] = False # don't use the in-built figure export/saving to disk functionality as we want to wrap the output figure with the Pdf saving, not write to a .png
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    ## TypeError: neuropy.utils.debug_helpers.safely_accepts_kwargs.<locals>._safe_kwargs_fn() got multiple values for keyword argument 'computation_config'
    for filter_name in curr_active_pipeline.filtered_session_names:
        print(f'filter_name: {filter_name}')
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.SPIKES_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.SPIKES_MAPS, fignum=active_identifying_ctx_string, **figure_format_config)) # works!
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.TUNING_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=active_identifying_ctx_string, **figure_format_config))
        for a_fig in out_fig_list:
            pdf.savefig(a_fig, transparent=True)

In [ ]:
%%capture
curr_display_function_name = '_display_decoder_result'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
    print(plots)
    # pdf.savefig(a_fig)

#### PyQtGraph-based plots:

#### PyQtGraph-based Pf2D Viewers:

In [ ]:
# 🟢✅ Nearly Completely Working - Needs subplot labels changed to match standardized matplotlib version, needs color scheme set consistently to matplotlib version, needs colorbars removed
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, add_bin_ticks, build_binned_imageItem
from neuropy.utils.matplotlib_helpers import _build_variable_max_value_label, enumTuningMap2DPlotMode, enumTuningMap2DPlotVariables, _determine_best_placefield_2D_layout, _scale_current_placefield_to_acceptable_range
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import display_all_pf_2D_pyqtgraph_binned_image_rendering

# NOTE FILTER SPECIFIC: active_config_name and active_pf_2D depend on active_config_name

## Get the figure_format_config from the figure_format_config widget:
active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='display_all_pf_2D_pyqtgraph_binned_image_rendering')
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
out_all_pf_2D_pyqtgraph_binned_image_fig = display_all_pf_2D_pyqtgraph_binned_image_rendering(active_pf_2D, figure_format_config)

In [ ]:
out_all_pf_2D_pyqtgraph_binned_image_fig.setWindowTitle(f'{active_identifying_display_ctx.get_description()}')

In [ ]:
images = active_one_step_decoder.ratemap.normalized_tuning_curves
images.shape # (66, 41, 63)

In [ ]:
# 🟢🚧🟨 Almost Working - Needs subplot labels changed from Cell[i] to the appropriate standardized titles. Needs other minor refinements.
# 🚧 pyqtplot_plot_image_array needs major improvements to achieve feature pairity with display_all_pf_2D_pyqtgraph_binned_image_rendering, so probably just use display_all_pf_2D_pyqtgraph_binned_image_rendering.  
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image_array

# Get the decoders from the computation result:       
# Get flat list of images:
images = active_one_step_decoder.ratemap.normalized_tuning_curves # (43, 63, 63)
occupancy = active_one_step_decoder.ratemap.occupancy

active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='pyqtplot_plot_image_array')
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
## Get final discription string:
active_identifying_ctx_string = active_identifying_display_ctx.get_description(separator='|')
print(f'active_identifying_ctx_string: {active_identifying_ctx_string}')

## Build the widget:
app, parent_root_widget, root_render_widget, plot_array, img_item_array, other_components_array = pyqtplot_plot_image_array(active_one_step_decoder.xbin, active_one_step_decoder.ybin, images, occupancy, 
                                                                        app=None, parent_root_widget=None, root_render_widget=None, max_num_columns=8)
parent_root_widget.show()
if master_dock_win is not None:
    # if there's an open master_dock_win, add this widget as a child dock
    master_dock_win.add_display_dock(identifier=active_identifying_ctx_string, widget=parent_root_widget, dockIsClosable=True)

#### Decoder Plots:

In [ ]:
# Must switch back to the interactive backend here for the interactive/animated decoder plots:
matplotlib.use('Qt5Agg')
# backend_qt5agg
import matplotlib.pyplot as plt
# plt.switch_backend('Qt5Agg')

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_animated_2D', active_config_name, variable_name='p_x_given_n')

In [ ]:
# ## MATPLOTLIB Imports:
# import matplotlib
# # configure backend here
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
# import matplotlib as mpl
## This plot looks phenominal, and the slider works!
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n') # NOW: TypeError: _temp_debug_two_step_plots_animated_imshow() missing 1 required positional argument: 'time_binned_position_df'

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n_and_x_prev')  # this one doesn't work!

In [ ]:
# Get the decoders from the computation result:
# active_one_step_decoder = computation_result.computed_data['pf2D_Decoder']
# active_two_step_decoder = computation_result.computed_data.get('pf2D_TwoStepDecoder', None)
# active_measured_positions = computation_result.sess.position.to_dataframe()

active_one_step_decoder # BayesianPlacemapPositionDecoder
active_two_step_decoder

## SAVE OUT THE RESULTS of the decoder:

In [ ]:
## PDF Output, NOTE this is single plot stuff: uses active_config_name
from matplotlib.backends import backend_pdf, backend_pgf, backend_ps
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

In [ ]:
## 2022-10-04 Modern Programmatic PDF outputs:
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices',  debug_print=False)
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='ripple', decoding_time_bin_size=0.02, debug_test_max_num_slices=128, debug_print=True)

In [ ]:
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='laps', debug_test_max_num_slices=128, debug_print=False)

In [ ]:
## TODO: trying to hack up 'replay's which aren't working because of the overlaps, and strangely my overlaps code doesn't seem to work

In [ ]:
sess.replay

In [ ]:
sess.replay.flat_replay_idx.unique()

In [ ]:
from neuropy.utils.efficient_interval_search import verify_non_overlapping, get_non_overlapping_epochs

active_filter_epochs = deepcopy(sess.replay) # epoch object
# filter_epochs.columns # ['epoch_id', 'rel_id', 'start', 'end', 'replay_r', 'replay_p', 'template_id', 'flat_replay_idx', 'duration']
if not 'stop' in active_filter_epochs.columns:
    active_filter_epochs['stop'] = active_filter_epochs['end'].copy()     # Make sure it has the 'stop' column which is expected as opposed to the 'end' column
default_figure_name = f'Replay'

In [ ]:
# TODO 2022-10-04 - CORRECTNESS - AssertionError: Intervals in start_stop_times_arr must be non-overlapping. I believe this is due to the stop values overlapping somewhere
print(f'active_filter_epochs: {active_filter_epochs}')
## HANDLE OVERLAPPING EPOCHS: Note that there is a problem that occurs here with overlapping epochs for laps. Below we remove any overlapping epochs and leave only the valid ones.
is_non_overlapping = get_non_overlapping_epochs(active_filter_epochs[['start','stop']].to_numpy()) # returns a boolean array of the same length as the number of epochs 
# Just drop the rows of the dataframe that are overlapping:
# active_filter_epochs = active_filter_epochs[is_non_overlapping, :]
active_filter_epochs = active_filter_epochs.loc[is_non_overlapping]
print(f'active_filter_epochs: {active_filter_epochs}')

In [ ]:
is_non_overlapping = get_non_overlapping_epochs(active_filter_epochs[['start','stop']].to_numpy()) # returns a boolean array of the same length as the number of epochs 
np.where(np.logical_not(is_non_overlapping))[0]

In [ ]:
verify_non_overlapping(start_stop_times_arr=active_filter_epochs[['start','stop']].to_numpy())

### 🔜 2022-08-10 👁️‍🗨️ NOW: Plot animal positions on the computed posteriors:
The process of plotting the animal position on the decoder plot needs to be refined. Currently it works by re-implementing 

🔜 NEXT STEP: TODO: Make a "Datasource" like approach perhaps to provide the actual animal position at each point in time?
🐞🔜 BUG TODO: Noticed that for Bapun Day5 data, it looks like the current position point is being plotted incorrectly (it doesn't even move across the space much)

In [ ]:
## Get the current positions at each of the time_window_centers:
# active_resampled_measured_positions
time_binned_pos_df = active_computed_data.extended_stats.time_binned_position_df
active_resampled_pos_df = time_binned_pos_df  # 1717 rows × 16 columns
active_resampled_measured_positions = active_resampled_pos_df[['x','y']].to_numpy() # The measured positions resampled (interpolated) at the window centers. 
# np.shape(active_resampled_measured_positions) # (1911, 2)
active_one_step_decoder.active_time_window_centers.shape # (1911,)
print(f'active_one_step_decoder.active_time_window_centers.shape: {active_one_step_decoder.active_time_window_centers.shape}')
# Note this has 2900 rows × 24 columns and active_one_step_decoder.active_time_window_centers.shape is (2892,) for some reason. Shouldn't they be the same?

In [ ]:
active_resampled_pos_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

from PendingNotebookCode import _temp_debug_two_step_plots_animated_imshow

# Get the decoders from the computation result:
# active_one_step_decoder = computation_result.computed_data['pf2D_Decoder']
# active_two_step_decoder = computation_result.computed_data.get('pf2D_TwoStepDecoder', None)
# active_measured_positions = computation_result.sess.position.to_dataframe()

def _debug_on_frame_update(new_frame_idx, ax):
    print(f'_debug_on_frame_update(new_frame_idx: {new_frame_idx}, ax: {ax})')
    pass

# active_resampled_pos_df = active_computed_data.extended_stats.time_binned_position_df  # 1717 rows × 16 columns

# Simple plot type 1:
# plotted_variable_name = kwargs.get('variable_name', 'p_x_given_n') # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
plotted_variable_name = 'p_x_given_n' # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
_temp_debug_two_step_plots_animated_imshow(active_one_step_decoder, active_two_step_decoder, active_computed_data.extended_stats.time_binned_position_df, variable_name=plotted_variable_name, update_callback_function=_debug_on_frame_update) # Works

In [ ]:
curr_display_function_name = '_display_spike_rasters_pyqtplot_2D'
curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
## Works, displays my velocity/density result for both 2D and 1D:
# out_plot_1D, out_plot_2D = curr_active_pipeline.display('_display_speed_vs_PFoverlapDensity_plots', active_config_name)
curr_display_function_name = '_display_speed_vs_PFoverlapDensity_plots'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
plots

In [ ]:
curr_display_function_name = '_display_placemaps_pyqtplot_2D'
out_plots = curr_active_pipeline.display(curr_display_function_name, filter_name, max_num_columns=8)    
out_plots[1].show()

In [ ]:
out_plots[1].show()

In [ ]:
out_plots[1].close()

In [ ]:
## KNOWN BAD, locks up, do not execute:
curr_display_function_name = 'display_firing_rate_trends'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)

In [ ]:
# a_plot = plots[0] # PlotWidget 
# a_plot_item = a_plot.plotItem # PlotItem
# a_plot.scene() # GraphicsScene
export_pyqtgraph_plot(plots[0])

### 3D (PyVista/Vedo/etc)-based plots:

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name) # this works now!

In [ ]:
display_dict = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', active_config_name) # does not work, missing color info?
iplapsDataExplorer = display_dict['iplapsDataExplorer']
# plotter is available at
p = display_dict['plotter']
iplapsDataExplorer

In [ ]:
# curr_kdiba_pipeline.display(DefaultDisplayFunctions._display_3d_interactive_custom_data_explorer, 'maze1') # works!
curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', 'maze1_PYR') # works!